**notebook für die definition von MLPs zur klassifikation mit merkmalsvektoren von dino (Merkmalsvekoren sind schon berechnet) + training von diesen + testen/metriken (für iNat19 Datensatz)**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
!mkdir /content/dataset #kopieren von daten aus google drive, weil es dann viel schneller im training geht
!cd dataset && mkdir features
!cp -R FEATURE_DIRECTORY /content/dataset/features

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class Seg_Dataset(Dataset):
  def __init__(self, annotations_file, dataset_dir, taxonomy):  #taxonomy must be: order family genus name (name ist für speziesebene)
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  
    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()

    def mapping(taxonomy):
      mapping = {}
      cat_ids = self.coco.getCatIds()
      categories = self.coco.loadCats(cat_ids)
      by_tax_cat = list(set([cat[taxonomy] for cat in categories]))
      by_tax_cat.sort()
      numb_cats = len(by_tax_cat)
      for i in range(numb_cats):
        for cat in categories:
          if cat[taxonomy] == by_tax_cat[i]:
            mapping[cat['id']] = i + 1
      return mapping
      
    self.cat_mapping = mapping(taxonomy)
    self.cat_len = len(self.cat_mapping)

  def __len__(self):
    return len(self.img_ids)

  def __getitem__(self, idx):
    img_id = self.img_ids[idx]
    img = self.coco.loadImgs(img_id)
    ann_ids = self.coco.getAnnIds(img[0]['id'])
    anns = self.coco.loadAnns(ann_ids)
    fileN = img[0]['file_name'].split('/')
    classN = fileN[1]
    direcN = fileN[2]
    imgN = fileN[3].split('.')[0]
    feature_path = self.dataset + '/features/' + classN + '/' + direcN + '/' + imgN + '_feature.pt'
    features = torch.load(feature_path, map_location=torch.device('cpu'))
    ground_truth = 0
    if len(anns) > 0:
      ground_truth = self.cat_mapping[anns[0]['category_id']]
    gt_vect = np.zeros(self.cat_len)
    gt_vect[ground_truth-1] = 1
    gt_vect = torch.tensor(gt_vect)

    return features, gt_vect

In [ ]:
dataset_path = '/content/dataset'

taxonomy = 'name' #order family genus name

trainset = Seg_Dataset('PATH_TO_TRAIN_ANNOTATION', dataset_path, taxonomy) 
testset = Seg_Dataset('PATH_TO_TEST_ANNOTATION', dataset_path, taxonomy) 

In [ ]:
import random
random.seed(7)
train_ids = np.arange(0,trainset.__len__(),1)
test_ids = np.arange(0,testset.__len__(),1)

Netzwerke. 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP_net(nn.Module):

    def __init__(self, input_size, output_size):
      super(MLP_net, self).__init__()

      self.lin1 = nn.Linear(input_size, input_size, bias=True)
      self.lin2 = nn.Linear(input_size, input_size, bias=True)
      self.lin2plus = nn.Linear(input_size, input_size, bias=True)
      #self.lin2plusplus = nn.Linear(input_size, input_size, bias=True)
      self.lin3 = nn.Linear(input_size, int(input_size/2), bias=True)
      #self.lin3plus = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      #self.lin3plus1 = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      self.lin4 = nn.Linear(int(input_size/2), output_size, bias=True)

    def forward(self, x):
      x = self.lin1(x)
      x = F.relu(x)
      x = self.lin2(x)
      x = F.relu(x)
      x = self.lin2plus(x)
      x = F.relu(x)
      #x = self.lin2plusplus(x)
      #x = F.relu(x)
      x = self.lin3(x)
      x = F.relu(x)
      #x = self.lin3plus(x)
      #x = F.relu(x)
      #x = self.lin3plus1(x)
      #x = F.relu(x)
      x = self.lin4(x)
      return x

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataset_path = '/content/dataset'

trainset = Seg_Dataset('PATH_TO_TRAINSET', dataset_path, taxonomy) #class order family genus name

batch_size = 32

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
print(trainset.cat_mapping)
print(trainset.cat_len) 

In [ ]:
net = MLP_net(768, trainset.cat_len)

net.to(device)

#PATH = 'MODEL_PATH.pth'
#net.load_state_dict(torch.load(PATH))
net.train()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.0001)

In [ ]:
#loss_graph = []

for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, mask = data
        inputs = inputs.to(device)
        numb_inst = inputs.shape[0] #number of actual batches. falls am ende des datensatzes nicht mehr die volle batchsize vorhanden ist
        mask = mask.to(device)
        mask = mask.type(torch.float32)
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = net(inputs)
        outputs = outputs.reshape((numb_inst ,175))# 175 ist anzahl an klassen auf speziesebene
        loss = criterion(outputs, mask)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        output_value = 500

        if i % output_value == output_value - 1: 
          print('[%d, %5d] loss: %.20f' %
              (epoch + 1, i + 1, running_loss / output_value))
          loss_graph.append(running_loss / output_value)
          running_loss = 0.0
          PATH = '/content/gdrive/MyDrive/models/nets/simple/' + str(i) + 'model_mlp.pth'
          #torch.save(net.state_dict(), PATH) #checkpoints entfernt, weil das nur viel langsamer wird

print('Finished Training')
PATH = 'MODEL_SAVE.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
import matplotlib.pyplot as plt

loss_arr = np.asarray(loss_graph)
x_val = np.arange(0, len(loss_graph), 1)

plt.plot(x_val, loss_arr)
plt.ylabel('loss')
plt.show()

# ab hier testing von anderem stuff


In [ ]:
drive.flush_and_unmount()

drive.mount('/content/gdrive')

In [ ]:
net.eval()
net.to(device)

In [ ]:
test_ids = np.arange(0,testset.__len__(),1)

Ygt = []
Ypred = []

net.eval()

import time
start = time.time() #zeit nehmen weil interessant

for i in test_ids:
  features, gt = testset.__getitem__(i)

  features = features.reshape(768)

  Ygt.append(np.asarray(gt, dtype=np.float32))
  features = features.to(device)
  pred = net(features)

  pred = F.softmax(pred, dim = 0)

  pred = pred.detach()
  pred = pred.to('cpu')

  Ypred.append(np.asarray(pred, dtype=np.float32))

end = time.time()
print((end-start)/423)

In [ ]:
YclassesGt = []
YclassesPred = []

#umrechnen der ausgabevektoren der länge 175 aud die tatsächliche klasse (höchsten wert im vektor finden)
for elem in Ygt:
  index = np.where(elem == 1)
  YclassesGt.append(int(index[0]))

for elem in Ypred:
  top = np.amax(elem)
  index = np.where(elem == top)
  YclassesPred.append(int(index[0]))

In [ ]:
Ypred = YclassesPred  #geht sonst kaputt
Ygt = YclassesGt

In [ ]:
from sklearn.metrics import confusion_matrix as confusion

matr = confusion(Ygt, Ypred)  #confusionmatrix berechnen

In [ ]:
norm_matrix = np.zeros(matr.shape)
#normalisieren
for i in range(matr.shape[0]):
  for j in range(matr.shape[1]):
    norm_matrix[i,j] = matr[i,j]/sum(matr[i,:])


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

#ausgabe der confusionmatrizen in schön
disp = ConfusionMatrixDisplay(matr)
disp.plot()
disp1 = ConfusionMatrixDisplay(norm_matrix)
disp1.plot()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score

#berechnen der verschiedenen metriken
accuracy = accuracy_score(Ygt, Ypred)
f1 = f1_score(Ygt, Ypred, average = 'macro')
precision = precision_score(Ygt, Ypred, average = 'macro')
recall = recall_score(Ygt, Ypred, average = 'macro')

In [ ]:
print(accuracy)
print(f1)
print(precision)
print(recall)

# PCA